In [44]:
import ee
import os
import geemap
import calendar

In [45]:
ee.Initialize()

In [46]:
roi =  ee.Geometry.Polygon(
        [[[36.050662943101294, -0.30239147953987083],
          [36.050662943101294, -0.42323864359236807],
          [36.12447733519114, -0.42323864359236807],
          [36.12447733519114, -0.30239147953987083]]])

In [47]:
def maskS2clouds(image):
    qa = image.select('QA60')
    #   Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
    return image.updateMask(mask).divide(10000);

In [48]:
def monthlyLake(month):
    start_date,end_date,_ = month
    dataset = (ee.ImageCollection('COPERNICUS/S2_SR').filterDate(start_date,end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',25))
    .filterBounds(roi)
    .map(maskS2clouds)
    .mean())
    return dataset

In [49]:
def get_start_end_dates(year,months):
    MONTHS = []
    
    for month in months:
        _,no_of_days = calendar.monthrange(YEAR, month)
        month_str = str(month)
        month_start = f'{YEAR}-{month_str.zfill(2)}-01'
        month_end = f'{YEAR}-{month_str.zfill(2)}-{no_of_days}'
        month_name = calendar.month_name[month]
        MONTHS.append((month_start,month_end,month_name))
        
    return MONTHS

In [53]:
YEAR = 2020
MONTHS = range(1,10)

months = get_start_end_dates(YEAR,MONTHS)

coImages = ee.ImageCollection([monthlyLake(month) for month in months])
month_names = [f'{month[2]} - {YEAR}' for month in months]

In [52]:
Map = geemap.Map()


vis = {
  "min": 0.0,
  "max": 0.3,
  "bands": ['B4', 'B3', 'B2'],
};

Map.addLayer(coImages.first().clip(roi),vis)

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [65]:
video_args = {
  'dimensions': 600,
  'region': roi,
  'framesPerSecond': 1,
  "min": 0.0,
  "max": 0.3,
  "bands": ['B8', 'B11', 'B4'],
}

work_dir = os.path.join(os.path.expanduser("~"), 'Downloads')
out_gif = os.path.join(work_dir, "test.gif")

geemap.download_ee_video(coImages, video_args, out_gif)

texted_gif = os.path.join(work_dir, "test_text.gif")

geemap.add_text_to_gif(out_gif,
                       texted_gif, 
                       xy=('2%', '1%'), 
                       font_size=24, 
                       font_color='#000', 
                       duration=1000,
                       text_sequence = month_names,
                       add_progress_bar=True)

geemap.show_image(texted_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/erickotenyo/Downloads/test.gif


Output()